In [10]:
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-5]  # n번째 레이어의 hidden states를 반환합니다.
        loss = outputs.loss
        return logits, loss, hidden_states

# 데이터 로드 및 전처리
data_A = pd.read_csv("random_500.csv")  # data set A 파일명에 맞게 수정
data_B = pd.read_csv("infected.csv")  # data set B 파일명에 맞게 수정
# 모델 저장 경로
model_path = "Pre-trained.pt"

# 중복된 환자 정보 제거
data_A_unique = data_A.drop_duplicates(subset="ID")

# X_train, Y_train 생성
X_train = []
Y_train = []

for index, row in data_A_unique.iterrows():
    patient_id = row["ID"]
    patient_info = [str(row[column]) for column in data_A.columns if column != "ID" and column != "DESCRIPTION"]
    symptoms = ", ".join(data_A[data_A["ID"] == patient_id]["DESCRIPTION"].tolist())
    combined_info = ", ".join(patient_info) + ", " + symptoms
    X_train.append(combined_info)
    if patient_id in data_B.values:
        Y_train.append(1)
    else:
        Y_train.append(0)

print("X_train\n", X_train[:10])
print("Y_train\n", Y_train[:10])

# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# 모델이 이미 저장되어 있는지 확인하고, 저장된 모델이 있으면 불러오고 없으면 새로운 모델 생성
if os.path.exists(model_path):
    # 저장된 모델이 있을 경우 불러오기
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    print("Pre-train model loaded.")
else:
    # 저장된 모델이 없을 경우 새로운 모델 생성
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    print("New model generated.")

# 입력 데이터를 BERT의 입력 형식으로 변환
max_len = 128  # 입력 시퀀스의 최대 길이

input_ids = []
attention_masks = []

for info in X_train:
    encoded_dict = tokenizer.encode_plus(
                        info,                         # 환자 정보 및 증상
                        add_special_tokens = True,    # [CLS], [SEP] 토큰 추가
                        max_length = max_len,         # 최대 길이 지정
                        pad_to_max_length = True,     # 패딩을 추가하여 최대 길이로 맞춤
                        return_attention_mask = True, # 어텐션 마스크 생성
                        return_tensors = 'pt',        # PyTorch 텐서로 반환
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y_train)

# 데이터셋 생성
dataset = TensorDataset(input_ids, attention_masks, labels)

# 전체 데이터셋을 훈련 데이터셋으로 사용
train_dataset = dataset

# 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# 모델을 GPU로 이동
model.to(device)

# 옵티마이저 및 학습률 설정
# 기본 학습률 : 2e-6
optimizer = torch.optim.AdamW(model.parameters(), lr=0)

# 에폭 설정
epochs = 3

# 학습 루프
for epoch in range(epochs):
    model.train()
    total_loss = 0
    hidden_states_list = []  # 각 배치의 hidden state를 저장할 리스트
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs[1]  # loss가 outputs의 두 번째 값입니다.
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Batch Loss: {loss.item()}')
        # hidden state를 저장합니다.
        hidden_states = outputs[2]
        hidden_states_list.append(hidden_states)

    # 각 배치의 hidden state를 합쳐서 CSV 파일로 저장합니다.
    hidden_states = torch.cat(hidden_states_list, dim=0)
    hidden_states = hidden_states[:, 0, :].cpu().detach().numpy()
    hidden_states_df = pd.DataFrame(hidden_states)
    hidden_states_df.to_csv(f"hidden_states_epoch{epoch + 1}.csv", index=False)  # 각 epoch마다 파일 이름을 다르게 지정합니다.

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss}')

# 모델 저장
torch.save(model.state_dict(), model_path)


X_train
 ["5/11/1967, nan, 999-83-3739, S99963041, X22481021X, Mrs., Hortense60, O'Hara248, nan, Spinka232, M, white, nonhispanic, F, Northborough  Massachusetts  US, 1040 Turner Knoll, Milford, Massachusetts, Worcester County, nan, 42.13519473, -71.50138606, 1137082.44, 9598.16, 11/12/2019, 1/21/2020, 803d9786-29a1-466c-9365-0205ea0a031c, 36971009.0, Sinusitis (disorder)", '11/19/1965, nan, 999-87-9895, S99998149, X88597197X, Mr., Waylon572, Reinger292, nan, nan, M, white, nonhispanic, M, Seekonk  Massachusetts  US, 895 Robel Light, Worcester, Massachusetts, Worcester County, 1604.0, 42.32627029, -71.79641283, 1134089.03, 5790.2, 5/7/2004, nan, ef7b5155-d4be-4b0f-98d4-1313f9b7e90b, 162864005.0, Body mass index 30+ - obesity (finding)', '1/20/1912, nan, 999-73-8598, S99979217, X22749225X, Mr., Hans694, Yost751, nan, nan, M, white, nonhispanic, M, Chelsea  Massachusetts  US, 158 Frami Drive Apt 55, Chelmsford, Massachusetts, Middlesex County, nan, 42.62575626, -71.41794612, 372357.04, 6

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Pre-train model loaded.
True
Epoch 1/3, Batch Loss: 0.17082533240318298
Epoch 1/3, Batch Loss: 0.24502494931221008
Epoch 1/3, Batch Loss: 0.23047764599323273
Epoch 1/3, Batch Loss: 0.15948261320590973
Epoch 1/3, Batch Loss: 0.2757938504219055
Epoch 1/3, Batch Loss: 0.16593442857265472
Epoch 1/3, Batch Loss: 0.12247272580862045
Epoch 1/3, Batch Loss: 0.15486426651477814
Epoch 1/3, Batch Loss: 0.20773708820343018
Epoch 1/3, Batch Loss: 0.18477430939674377
Epoch 1/3, Batch Loss: 0.15189668536186218
Epoch 1/3, Batch Loss: 0.10413150489330292
Epoch 1/3, Batch Loss: 0.2195568084716797
Epoch 1/3, Batch Loss: 0.13240273296833038
Epoch 1/3, Batch Loss: 0.19105307757854462
Epoch 1/3, Batch Loss: 0.062119368463754654
Epoch 1/3, Batch Loss: 0.11713354289531708
Epoch 1/3, Batch Loss: 0.31479838490486145
Epoch 1/3, Batch Loss: 0.08140704035758972
Epoch 1/3, Batch Loss: 0.12966172397136688
Epoch 1/3, Batch Loss: 0.1426646113395691
Epoch 1/3, Batch Loss: 0.3265485465526581
Epoch 1/3, Batch Loss: 0.188

In [ ]:
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch

class CustomBertForSequenceClassification(BertForSequenceClassification):
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        labels=None,
        output_hidden_states=True
    ):
        outputs = super().forward(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            labels=labels,
            output_hidden_states=output_hidden_states
        )
        logits = outputs.logits
        hidden_states = outputs.hidden_states[-5]  # n번째 레이어의 hidden states를 반환합니다.
        loss = outputs.loss
        return logits, loss, hidden_states

# 데이터 로드
data_A = pd.read_csv("output1.csv")  # data set A 파일명에 맞게 수정
data_B = pd.read_csv("infected.csv")  # data set B 파일명에 맞게 수정
model_path = "Pre-trained.pt"

# X_train, Y_train 생성
X_train = []
Y_train = []

for index, row in data_A.iterrows():  # 중복 제거를 하지 않고 원본 데이터 사용
    patient_id = row["ID"]
    patient_info = [str(row[column]) for column in data_A.columns if column != "ID" and column != "DESCRIPTION"]
    symptoms = ", ".join(data_A[data_A["ID"] == patient_id]["DESCRIPTION"].tolist())
    combined_info = ", ".join(patient_info) + ", " + symptoms
    X_train.append(combined_info)
    if patient_id in data_B.values:
        Y_train.append(1)
    else:
        Y_train.append(0)

print("X_train\n", X_train[:10])
print("Y_train\n", Y_train[:10])

# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# 모델이 이미 저장되어 있는지 확인하고, 저장된 모델이 있으면 불러오고 없으면 새로운 모델 생성
if os.path.exists(model_path):
    # 저장된 모델이 있을 경우 불러오기
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.load_state_dict(torch.load(model_path))
    print("Pre-trained model loaded.")
else:
    # 저장된 모델이 없을 경우 새로운 모델 생성
    model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    print("New model generated.")

# 입력 데이터를 BERT의 입력 형식으로 변환
max_len = 128  # 입력 시퀀스의 최대 길이

input_ids = []
attention_masks = []

for info in X_train:
    encoded_dict = tokenizer.encode_plus(
                        info,                         # 환자 정보 및 증상
                        add_special_tokens = True,    # [CLS], [SEP] 토큰 추가
                        max_length = max_len,         # 최대 길이 지정
                        pad_to_max_length = True,     # 패딩을 추가하여 최대 길이로 맞춤
                        return_attention_mask = True, # 어텐션 마스크 생성
                        return_tensors = 'pt',        # PyTorch 텐서로 반환
                   )
    
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(Y_train)

# 데이터셋 생성
dataset = TensorDataset(input_ids, attention_masks, labels)

# 전체 데이터셋을 훈련 데이터셋으로 사용
train_dataset = dataset

# 데이터로더 생성
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

# 모델을 GPU로 이동
model.to(device)

# 옵티마이저 및 학습률 설정
# 기본 학습률 : 2e-6
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-6)

# 에폭 설정
epochs = 10

# 학습 루프
for epoch in range(epochs):
    model.train()
    total_loss = 0
    hidden_states_list = []  # 각 배치의 hidden state를 저장할 리스트
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs[1]  # loss가 outputs의 두 번째 값입니다.
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        print(f'Epoch {epoch + 1}/{epochs}, Batch Loss: {loss.item()}')
            # hidden state를 저장합니다.
        #hidden_states = outputs[2]
        #hidden_states_list.append(hidden_states)

        # 각 배치의 hidden state를 합쳐서 CSV 파일로 저장합니다.
    #hidden_states = torch.cat(hidden_states_list, dim=0)
    #hidden_states = hidden_states[:, 0, :].cpu().detach().numpy()
    #hidden_states_df = pd.DataFrame(hidden_states)
    #hidden_states_df.to_csv(f"hidden_states_epoch{epoch + 1}.csv", index=False)  # 각 epoch마다 파일 이름을 다르게 지정합니다.

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss}')

# 모델 저장
torch.save(model.state_dict(), model_path)


X_train
 ["5/11/1967, nan, 999-83-3739, S99963041, X22481021X, Mrs., Hortense60, O'Hara248, nan, Spinka232, M, white, nonhispanic, F, Northborough  Massachusetts  US, 1040 Turner Knoll, Milford, Massachusetts, Worcester County, nan, 42.13519473, -71.50138606, 1137082.44, 9598.16, 11/12/2019, 1/21/2020, 803d9786-29a1-466c-9365-0205ea0a031c, 36971009.0, Sinusitis (disorder), Body mass index 30+ - obesity (finding), Prediabetes, Chronic sinusitis (disorder)", '11/19/1965, nan, 999-87-9895, S99998149, X88597197X, Mr., Waylon572, Reinger292, nan, nan, M, white, nonhispanic, M, Seekonk  Massachusetts  US, 895 Robel Light, Worcester, Massachusetts, Worcester County, 1604.0, 42.32627029, -71.79641283, 1134089.03, 5790.2, 5/7/2004, nan, ef7b5155-d4be-4b0f-98d4-1313f9b7e90b, 162864005.0, Body mass index 30+ - obesity (finding), Vomiting symptom (finding), Bacterial infectious disease (disorder), Pneumonia (disorder), Hypertension, Chronic sinusitis (disorder), Fever (finding), Hyperlipidemia, Hy

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\MCC\anaconda3\envs\biotf\lib\site-packages\transformers\tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to 

New model generated.
True
Epoch 1/10, Batch Loss: 0.6627999544143677
Epoch 1/10, Batch Loss: 0.6521687507629395
Epoch 1/10, Batch Loss: 0.6383733153343201
Epoch 1/10, Batch Loss: 0.6632502675056458
Epoch 1/10, Batch Loss: 0.6625816226005554
Epoch 1/10, Batch Loss: 0.6352531313896179
Epoch 1/10, Batch Loss: 0.6540289521217346
Epoch 1/10, Batch Loss: 0.5975910425186157
Epoch 1/10, Batch Loss: 0.5182635188102722
Epoch 1/10, Batch Loss: 0.5916292667388916
Epoch 1/10, Batch Loss: 0.5372036099433899
Epoch 1/10, Batch Loss: 0.46049416065216064
Epoch 1/10, Batch Loss: 0.6505278944969177
Epoch 1/10, Batch Loss: 0.43817150592803955
Epoch 1/10, Batch Loss: 0.6276784539222717
Epoch 1/10, Batch Loss: 0.4209035038948059
Epoch 1/10, Batch Loss: 0.5010300874710083
Epoch 1/10, Batch Loss: 0.5159713625907898
Epoch 1/10, Batch Loss: 0.5134479403495789
Epoch 1/10, Batch Loss: 0.48386478424072266
Epoch 1/10, Batch Loss: 0.6082154512405396
Epoch 1/10, Batch Loss: 0.3859703242778778
Epoch 1/10, Batch Loss: 0

Epoch 1/10, Batch Loss: 0.3498584032058716
Epoch 1/10, Batch Loss: 0.4673073887825012
Epoch 1/10, Batch Loss: 0.1789143830537796
Epoch 1/10, Batch Loss: 0.5453346371650696
Epoch 1/10, Batch Loss: 0.34208807349205017
Epoch 1/10, Batch Loss: 0.23585689067840576
Epoch 1/10, Batch Loss: 0.4334607422351837
Epoch 1/10, Batch Loss: 0.5746684074401855
Epoch 1/10, Batch Loss: 0.3309243321418762
Epoch 1/10, Batch Loss: 0.27008676528930664
Epoch 1/10, Batch Loss: 0.22530916333198547
Epoch 1/10, Batch Loss: 0.27296632528305054
Epoch 1/10, Batch Loss: 0.403519868850708
Epoch 1/10, Batch Loss: 0.35740113258361816
Epoch 1/10, Batch Loss: 0.38906118273735046
Epoch 1/10, Batch Loss: 0.4175785481929779
Epoch 1/10, Batch Loss: 0.38001716136932373
Epoch 1/10, Batch Loss: 0.205058291554451
Epoch 1/10, Batch Loss: 0.3802506923675537
Epoch 1/10, Batch Loss: 0.23237921297550201
Epoch 1/10, Batch Loss: 0.22522705793380737
Epoch 1/10, Batch Loss: 0.3438342809677124
Epoch 1/10, Batch Loss: 0.2238709032535553
Epo

Epoch 1/10, Batch Loss: 0.2896566689014435
Epoch 1/10, Batch Loss: 0.23159554600715637
Epoch 1/10, Batch Loss: 0.13376061618328094
Epoch 1/10, Batch Loss: 0.0948353260755539
Epoch 1/10, Batch Loss: 0.27694976329803467
Epoch 1/10, Batch Loss: 0.2392335832118988
Epoch 1/10, Batch Loss: 0.0990254357457161
Epoch 1/10, Batch Loss: 0.40880000591278076
Epoch 1/10, Batch Loss: 0.19035246968269348
Epoch 1/10, Batch Loss: 0.3012046217918396
Epoch 1/10, Batch Loss: 0.2815692722797394
Epoch 1/10, Batch Loss: 0.0796794593334198
Epoch 1/10, Batch Loss: 0.136541485786438
Epoch 1/10, Batch Loss: 0.362741082906723
Epoch 1/10, Batch Loss: 0.17732977867126465
Epoch 1/10, Batch Loss: 0.1489259898662567
Epoch 1/10, Batch Loss: 0.09270945191383362
Epoch 1/10, Batch Loss: 0.45481613278388977
Epoch 1/10, Batch Loss: 0.26320797204971313
Epoch 1/10, Batch Loss: 0.10464777797460556
Epoch 1/10, Batch Loss: 0.10816624015569687
Epoch 1/10, Batch Loss: 0.10848790407180786
Epoch 1/10, Batch Loss: 0.2772783637046814
E

...

Epoch 7/10, Batch Loss: 0.005472760181874037
Epoch 7/10, Batch Loss: 0.10253190994262695
Epoch 7/10, Batch Loss: 0.024850990623235703
Epoch 7/10, Batch Loss: 0.12003758549690247
Epoch 7/10, Batch Loss: 0.03172507882118225
Epoch 7/10, Batch Loss: 0.10916347056627274
Epoch 7/10, Batch Loss: 0.006291496567428112
Epoch 7/10, Batch Loss: 0.04436373710632324
Epoch 7/10, Batch Loss: 0.12067174166440964
Epoch 7/10, Batch Loss: 0.005066723097115755
Epoch 7/10, Batch Loss: 0.004821099806576967
Epoch 7/10, Batch Loss: 0.16624225676059723
Epoch 7/10, Batch Loss: 0.01099677849560976
Epoch 7/10, Batch Loss: 0.009312818758189678
Epoch 7/10, Batch Loss: 0.042441632598638535
Epoch 7/10, Batch Loss: 0.020555585622787476
Epoch 7/10, Batch Loss: 0.04222936928272247
Epoch 7/10, Batch Loss: 0.04753271862864494
Epoch 7/10, Batch Loss: 0.019361473619937897
Epoch 7/10, Batch Loss: 0.0651157945394516
Epoch 7/10, Batch Loss: 0.0055516380816698074
Epoch 7/10, Batch Loss: 0.0462530143558979
Epoch 7/10, Batch Loss:

Epoch 7/10, Batch Loss: 0.008716622367501259
Epoch 7/10, Batch Loss: 0.06434699147939682
Epoch 7/10, Batch Loss: 0.02270875871181488
Epoch 7/10, Batch Loss: 0.012122685089707375
Epoch 7/10, Batch Loss: 0.012089739553630352
Epoch 7/10, Batch Loss: 0.031502097845077515
Epoch 7/10, Batch Loss: 0.014033331535756588
Epoch 7/10, Batch Loss: 0.019991278648376465
Epoch 7/10, Batch Loss: 0.008040143176913261
Epoch 7/10, Batch Loss: 0.007766212802380323
Epoch 7/10, Batch Loss: 0.0144104128703475
Epoch 7/10, Batch Loss: 0.1328020840883255
Epoch 7/10, Batch Loss: 0.020559288561344147
Epoch 7/10, Batch Loss: 0.02111821249127388
Epoch 7/10, Batch Loss: 0.2024056762456894
Epoch 7/10, Batch Loss: 0.02081085368990898
Epoch 7/10, Batch Loss: 0.028224138543009758
Epoch 7/10, Batch Loss: 0.13102462887763977
Epoch 7/10, Batch Loss: 0.23992735147476196
Epoch 7/10, Batch Loss: 0.015978310257196426
Epoch 7/10, Batch Loss: 0.037361130118370056
Epoch 7/10, Batch Loss: 0.22600378096103668
Epoch 7/10, Batch Loss:

Epoch 7/10, Batch Loss: 0.01678438112139702
Epoch 7/10, Batch Loss: 0.07710197567939758
Epoch 7/10, Batch Loss: 0.07715163379907608
Epoch 7/10, Batch Loss: 0.22146646678447723
Epoch 7/10, Batch Loss: 0.009116385132074356
Epoch 7/10, Batch Loss: 0.006648646201938391
Epoch 7/10, Batch Loss: 0.054149042814970016
Epoch 7/10, Batch Loss: 0.010903341695666313
Epoch 7/10, Batch Loss: 0.010317510925233364
Epoch 7/10, Batch Loss: 0.08297517150640488
Epoch 7/10, Batch Loss: 0.004058211110532284
Epoch 7/10, Batch Loss: 0.013730666600167751
Epoch 7/10, Batch Loss: 0.05100499466061592
Epoch 7/10, Batch Loss: 0.02065407484769821
Epoch 7/10, Batch Loss: 0.0321236327290535
Epoch 7/10, Batch Loss: 0.010617807507514954
Epoch 7/10, Batch Loss: 0.008327997289597988
Epoch 7/10, Batch Loss: 0.22572962939739227
Epoch 7/10, Batch Loss: 0.21928146481513977
Epoch 7/10, Batch Loss: 0.06413646787405014
Epoch 7/10, Batch Loss: 0.005543037783354521
Epoch 7/10, Batch Loss: 0.043209172785282135
Epoch 7/10, Batch Loss

Epoch 7/10, Batch Loss: 0.1323067545890808
Epoch 7/10, Batch Loss: 0.016103094443678856
Epoch 7/10, Batch Loss: 0.09252555668354034
Epoch 7/10, Batch Loss: 0.05838711932301521
Epoch 7/10, Batch Loss: 0.06168997287750244
Epoch 7/10, Batch Loss: 0.13057459890842438
Epoch 7/10, Batch Loss: 0.036863721907138824
Epoch 7/10, Batch Loss: 0.016408156603574753
Epoch 7/10, Batch Loss: 0.21811719238758087
Epoch 7/10, Batch Loss: 0.001369310775771737
Epoch 7/10, Batch Loss: 0.009922951459884644
Epoch 7/10, Batch Loss: 0.003897759597748518
Epoch 7/10, Batch Loss: 0.010707278735935688
Epoch 7/10, Batch Loss: 0.11686035990715027
Epoch 7/10, Batch Loss: 0.015369663946330547
Epoch 7/10, Batch Loss: 0.012967933900654316
Epoch 7/10, Batch Loss: 0.14827711880207062
Epoch 7/10, Batch Loss: 0.021658075973391533
Epoch 7/10, Batch Loss: 0.045074090361595154
Epoch 7/10, Batch Loss: 0.04762722924351692
Epoch 7/10, Batch Loss: 0.028996320441365242
Epoch 7/10, Batch Loss: 0.015288006514310837
Epoch 7/10, Batch Lo

Epoch 7/10, Batch Loss: 0.03906158730387688
Epoch 7/10, Batch Loss: 0.031060872599482536
Epoch 7/10, Batch Loss: 0.037931136786937714
Epoch 7/10, Batch Loss: 0.026916038244962692
Epoch 7/10, Batch Loss: 0.1291418820619583
Epoch 7/10, Batch Loss: 0.03695700690150261
Epoch 7/10, Batch Loss: 0.038786809891462326
Epoch 7/10, Batch Loss: 0.006793799810111523
Epoch 7/10, Batch Loss: 0.008818880654871464
Epoch 7/10, Batch Loss: 0.0288409236818552
Epoch 7/10, Batch Loss: 0.018632279708981514
Epoch 7/10, Batch Loss: 0.00810044165700674
Epoch 7/10, Batch Loss: 0.044892050325870514
Epoch 7/10, Batch Loss: 0.37977707386016846
Epoch 7/10, Batch Loss: 0.011229017749428749
Epoch 7/10, Batch Loss: 0.00702503090724349
Epoch 7/10, Batch Loss: 0.002686902182176709
Epoch 7/10, Batch Loss: 0.02500814013183117
Epoch 7/10, Batch Loss: 0.17818692326545715
Epoch 7/10, Batch Loss: 0.028160832822322845
Epoch 7/10, Batch Loss: 0.24701707065105438
Epoch 7/10, Batch Loss: 0.03161904960870743
Epoch 7/10, Batch Loss:

In [37]:
# 추론 정확도 계산 (수정)
def calculate_accuracy(client_data, server_data, top_n_indices):
    correct_count = 0
    for i in range(len(client_data)):
        if i in top_n_indices[i]:
            correct_count += 1
    accuracy = correct_count / len(client_data)
    return accuracy

# 파일 경로
client_files = ['Client-trained_smashed_data_epoch1.csv', 'Client-trained_smashed_data_epoch2.csv', 'Client-trained_smashed_data_epoch3.csv']
server_files = ['Dictionary_smashed_data_epoch1.csv', 'Dictionary_smashed_data_epoch2.csv', 'Dictionary_smashed_data_epoch3.csv']

# 클라이언트 데이터 불러오기
client_data = np.concatenate([load_data(file) for file in client_files], axis=0)

# 서버 데이터 불러오기
server_data = np.concatenate([load_data(file) for file in server_files], axis=0)
print(client_data.shape)
print(server_data.shape)
# 정확도 계산을 위한 작업
total_correct_count = 0
total_clients = client_data.shape[0]

# 클라이언트 데이터마다 상위 5개의 추론 결과를 계산하고 정확도 측정
for i in range(total_clients):
    # 클라이언트 데이터 한 개에 대한 상위 5개의 추론 결과 계산
    client_single = client_data[i:i+1]
    top_n_indices_single = top_n_inference(client_single, server_data)
    # 클라이언트 데이터가 서버 데이터의 상위 5개 안에 포함되는지 확인
    if i in top_n_indices_single[0]:
        total_correct_count += 1

# 정확도 계산
accuracy = total_correct_count / total_clients
print("Top@5 Accuracy:", accuracy)


(303, 768)
(1458, 768)
Top@5 Accuracy: 0.0033003300330033004
